# `df.style`

Now yet merged into master, need this branch:

https://github.com/pydata/pandas/pull/10250

In [25]:
import pandas as pd
import numpy as np

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan

In [26]:
df.style

In [27]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val < 0 else 'black'
    return 'color: %s' % color

In [28]:
df.style.applymap(color_negative_red)

In [29]:
def highlight_max(s):
    '''
    highlight the maximum in a Series
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [30]:
df.style.apply(highlight_max)

In [31]:
(df
   .style
   .applymap(color_negative_red)
   .apply(highlight_max)
 )

In [13]:
df.style.applymap(color_negative_red,
                  subset=pd.IndexSlice[2:5, ['B', 'D']])

In [14]:
df.style.highlight_null(null_color='red')

In [32]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
df.style.background_gradient(cmap=cm)

In [16]:
df.style.bar(subset=['A', 'B'], color='#7F7FFF')

In [34]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover", props=[("background-color", "%s" % hover_color)])

styles = [hover(), dict(selector="th", props=[("font-size", "150%"),
                                              ("text-align", "center")])]
html = df.style.render(table_styles=styles, caption="Hover to highlight.")
HTML(html)

In [21]:
# https://developer.mozilla.org/en-US/docs/Web/CSS/animation#Cylon_Eye
# no animation yet :(

def cylon(s):
    tpl = """
  background-color: red;
  background-image: -webkit-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:    -moz-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:      -o-linear-gradient(    left, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  background-image:         linear-gradient(to right, rgba( 0,0,0,0.9 ) 25%, rgba( 0,0,0,0.1 ) 50%, rgba( 0,0,0,0.9 ) 75%);
  color: white;
  height: 100%;
  width: 20"""
    return pd.Series([tpl for i in s], index=s.index, name=s.name)

In [22]:
df.style.apply(cylon)

In [23]:
from IPython.html import widgets
@widgets.interact
def f(h_neg=(0, 359, 1), h_pos=(0, 359), s=(0., 99.9), l=(0., 99.9)):
    return df.style.background_gradient(
        cmap=sns.palettes.diverging_palette(h_neg=h_neg, h_pos=h_pos, s=s, l=l,
                                            as_cmap=True)
    )